<a href="https://colab.research.google.com/github/DamiamAlfaro/Accuracy/blob/main/DIRSearchResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import openpyxl
import re
from openpyxl.styles import numbers
from google.colab import files

#upload = files.upload()

In [3]:
"""
Initial Raw Text File: Breaking down the file by project
"""
def ReadingFile(self):
  with open(self,"r") as file:
    read = file.read()

  projects = read.split("Project Number:")
  return projects

In [4]:
"""
Breaking down each project into attributes:
I.e. Address, Name, Number, Amount, etc.
"""
def Stratifying(self):
  attributes = []
  for attribute in self:
    attributes.append(attribute.split("\n"))
  return attributes

In [5]:
"""
UglyList: just a list with every single
attribute within projects in no order.
"""
def UglyList(self):
  uglyList = []
  for i in self:
    for j in i:
      uglyList.append(j)

  newList = [item for item in uglyList if item != ""]
  return newList

In [6]:
"""
Identifying the subcontractors: analyzing
patterns in numbers and lengths of strings.
"""
entities = []
def Entities(self):
  for sub in self:
    if sub[0].isdigit():
      entitiesInvolved = sub.split("\t")
      if len(entitiesInvolved) > 1:
        entities.append(entitiesInvolved)
      else:
        continue
    else:
      continue

  return entities


In [7]:
"""
A literal version of all entities,
basically a list containing all entities
and their respective ID numbers.
"""
def EntitiesCleaned(self):
  entitiesCleaned = []
  for i in self:
    for j in i:
      entitiesCleaned.append(j)
  return entitiesCleaned

In [59]:
"""
We want to correlate the amounts with the subcontractor
entities, but the thing is that they are unorganized,
we want to put them together so the lengths correlate.
"""
def ReconfiguringEntities(self):
    collecting = False
    start_marker = "PWCR/CSLB/Lic\tName\tPWCR/CSLB/Lic\tName"
    end_marker = "DIR Project"
    collected_groups = []
    current_group = []

    for item in self:
      if item == start_marker:
          collecting = True
          current_group = []
          continue

      if collecting:
          if item == end_marker:
              current_group.append(item)
              collected_groups.append(current_group)
              collecting = False
          else:
              current_group.append(item)

    if collecting:
        collected_groups.append(current_group)

    return collected_groups





In [69]:
"""
Takes stratified entities list from
above and returns a clean one with
entities correlated with their respective
project.
"""
def CorrelatedEntities(self):
  stratifyingEntitiesList = [[item for item in sublist if "\t" in item]
                       for sublist in self]

  cleanEntitiesList = [[item2.split("\t") for item2 in sublist2]
                       for sublist2 in stratifyingEntitiesList]


  return cleanEntitiesList

In [9]:
"""
Getting Amount of projects: we need to know
where did the entities involved helped the GC.
"""
def ProjectAmount(self):
  amounts = []
  for attribute in range(len(self)):
    if self[attribute] == "Amount:":
      amount1 = self[attribute+1].split("\t")
      amount2 = amount1[0][1:]
      amount3 = amount2.replace(",","")
      amounts.append(float(amount3))
  return amounts

In [76]:
"""

NOT IN USE

Excel allocation: Allocating all to excel for
visual analysis.
"""
def ExcelEntityAllocation(self):
  workbook = openpyxl.Workbook()
  sheet = workbook.active
  sheet.title = "DictionaryData"
  row_a = 1
  row_b = 1

  for index, item in enumerate(self):
    if (index % 2) == 0:
        sheet.cell(row=row_a, column=1, value=item)
        row_a += 1
    else:
        sheet.cell(row=row_b, column=2, value=item)
        row_b += 1

  #workbook.save("Entities.xlsx")
  #files.download("Entities.xlsx")


In [11]:
"""
NOT IN USE

Allocating Amounts: Eventually corralting them
with the other attributes.

"""
def AmountAllocation(self):
  workbook = openpyxl.Workbook()
  sheet = workbook.active
  sheet.title = "AmountData"
  currency_format = numbers.BUILTIN_FORMATS[44]

  for index, value in enumerate(self, start=1):
    cell = sheet.cell(row=index, column=1, value=value)
    cell.number_format = currency_format

  # Save the workbook
  #workbook.save('FormattedCurrencyExcel.xlsx')
  #files.download('FormattedCurrencyExcel.xlsx')

In [81]:
"""

NOT IN USE

We want to find patterns in order to
collect accuratly each attribute, therefore
we need to have them in an unorganized
list first; just as as all data is at
the beginning by the way, the cunning trait
here is to organize it.
"""
def AttributesAllocation(self):
  workbook = openpyxl.Workbook()
  sheet = workbook.active
  sheet.title = "AttributesData"
  for index, value in enumerate(self,start=1):
    cell = sheet.cell(row=index,column=1, value=value)

  workbook.save("AttributeAllocation.xlsx")
  files.download("AttributeAllocation.xlsx")

In [13]:
"""
Luckily, we found that we can collect
three attributes at a time: DIR No,
Awarding Bodies, and Name of the Project.
"""
def DirAwardingBodiesName(self):
  DirAndAwardingBodies = []
  workbook = openpyxl.Workbook()
  sheet = workbook.active
  sheet.tile = "DIRABNamesData"
  for index in self:
    if "ID:" in index:
      stratification = index.split("\t")
      DirAndAwardingBodies.append(stratification)
  return DirAndAwardingBodies


In [19]:
"""
We want to know which projects are we
talking about, right?
"""
def LocationsAllocation(self):
  Locations = []
  workbook = openpyxl.Workbook()
  sheet = workbook.active
  sheet.title = "LocationsData"
  for index in self:
    if "Advertised:" in index:
      Locations.append(index.split("\t")[-2])

  return Locations

In [79]:
"""
Hitting DIR Numbers, Awarding bodies, Project
Names, Amounts, and Locations with one shot.
"""
def FiveBirdsOneHit(self,locations,amounts,entities):
  majorList = [self,locations,amounts]
  workbook = openpyxl.Workbook()
  sheet = workbook.active
  sheet.title = "HitData"
  currency_format = numbers.BUILTIN_FORMATS[44]
  DIRNumber, amounts, locations = majorList[0], majorList[1], majorList[2]

  for index, person in enumerate(DIRNumber):
    sheet.cell(row=index + 1, column=1, value=person[0])
    sheet.cell(row=index + 1, column=2, value=person[1])
    sheet.cell(row=index + 1, column=3, value=person[2])

    if index < len(amounts):
        sheet.cell(row=index + 1, column=4, value=amounts[index]
                   ).number_format = currency_format

    if index < len(locations):
        sheet.cell(row=index + 1, column=5, value=locations[index])

    if index < len(entities):
      combined_entities = []
      for sublist in entities[index]:
        paired_entities = ["{} {}".format(sublist[i],
                                          sublist[i+1]) if
                           i + 1 < len(sublist) else sublist[i]
                           for i in range(0, len(sublist), 2)]
        combined_entities.extend(paired_entities)
      for sub_index,entity in enumerate(combined_entities):
        sheet.cell(row=index + 1, column=6 + sub_index, value=entity)

  workbook.save("FiveHitsOneStone.xlsx")
  files.download("FiveHitsOneStone.xlsx")


In [80]:
initialFile = "Testing-AllBirdsHit.txt"

"""
Defines variables for the usable functions.
"""
brokenDownProjects1 = Stratifying(ReadingFile(initialFile))
AllAttributesList1 = UglyList(brokenDownProjects1)

"""
Attains all necessary attributes for
research: Amounts, Locations, and Correlated
Entities, Awarding Bodies, Project IDs, and
Project Names.
"""
Amounts1 = ProjectAmount(AllAttributesList1)
DAN1 = DirAwardingBodiesName(AllAttributesList1)
Locations1 = LocationsAllocation(AllAttributesList1)
EntitiesCorrelated = CorrelatedEntities(
    ReconfiguringEntities(AllAttributesList1))

FiveBirdsOneHit(DAN1,Amounts1,Locations1,EntitiesCorrelated)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>